# Module 5 - Lab 1 - Deploy LangGraph Dev
> <span style="color:red">**WARNING** must run on local env (not github codespaces)</span>
> - Step 6 will 
>   - start LangGraph Server API dev on localhost 
>   - spawn a browser tab for Studio UI, eg.
>       - 🚀 API: http://127.0.0.1:2024
>       - 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
>       - 📚 API Docs: http://127.0.0.1:2024/docs
>   - Appears this is not possible in github codespaces

## Pre-requisites
- Python virtual environment
- ollama installed and running (optional if using local LLM)
    - https://ollama.com/download

In [1]:
# Install packages for deployment and SDK
!pip install -U langchain langchain-openai langchain-community langgraph "langgraph-cli[inmem]" langgraph-sdk

  Attempting uninstall: langgraph-sdk
    Found existing installation: langgraph-sdk 0.1.56
    Uninstalling langgraph-sdk-0.1.56:
      Successfully uninstalled langgraph-sdk-0.1.56
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.3.8
    Uninstalling langgraph-0.3.8:
      Successfully uninstalled langgraph-0.3.8

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## Step 1 - Use LangGraph CLI to create template app

In [2]:
# view the LangGraph CLI help
!langgraph --help

Usage: langgraph [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  build       📦 Build LangGraph API server Docker image.
  dev         🏃‍♀️‍➡️ Run LangGraph API server in development mode with...
  dockerfile  🐳 Generate a Dockerfile for the LangGraph API server, with...
  new         🌱 Create a new LangGraph project from a template.
  up          🚀 Launch LangGraph API server.


In [3]:
# view the LangGraph CLI new command help
!langgraph new --help

Usage: langgraph new [OPTIONS] [PATH]

  🌱 Create a new LangGraph project from a template.

Options:
  --template TEXT  The name of the template to use. Available options: new-
                   langgraph-project-python new-langgraph-project-js react-
                   agent-python react-agent-js memory-agent-python memory-
                   agent-js retrieval-agent-python retrieval-agent-js data-
                   enrichment-agent-python data-enrichment-agent-js
  --help           Show this message and exit.


In [4]:
# Clean up prior to generating
!rm -rf chat_app

In [5]:
# Use LangGraph CLI to generate template for:
# - ReAct agent for Python 
# - in chat_app directory
!langgraph new --template react-agent-python chat_app

📥 Attempting to download repository as a ZIP archive...
URL: https://github.com/langchain-ai/react-agent/archive/refs/heads/main.zip
✅ Downloaded and extracted repository to /Volumes/CrucialSSD/Software/Developing_LLM_Applications/notebooks/chat_app
🎉 New project created at /Volumes/CrucialSSD/Software/Developing_LLM_Applications/notebooks/chat_app


## Step 2 - Install as package in editable mode

In [6]:
%cd chat_app 
!pip install -e .

/Volumes/CrucialSSD/Software/Developing_LLM_Applications/notebooks/chat_app
Obtaining file:///Volumes/CrucialSSD/Software/Developing_LLM_Applications/notebooks/chat_app
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for react-agent (pyproject.toml) ... done
  Created wheel for react-agent: filename=react_agent-0.0.1-0.editable-py3-none-any.whl size=6976 sha256=757c0a0d720241d594d6db9c96904870c01251ed7edf74d3bbafb9902196ffe7
  Stored in directory: /private/var/folders/jw/ft66857d2c13dsmc9514zfkw0000gn/T/pip-ephem-wheel-cache-qmtuv3jo/wheels/f2/8c/7b/5497d9f64ff9dad6b636bb3fdf1e57bf9081ba34b7aca11a27
Successfully built react-agent
  Attempting uninstall: react-agent
    Found existing installation: react-agent 0.0.1
    Uninstalling react-agent-0.0.1:
      Successfully uninstalled react-agent-0.0.1

[notice] 

In [7]:
# Verify package installed as editable mode
!pip list | grep react-agent

react-agent              0.0.1       /Volumes/CrucialSSD/Software/Developing_LLM_Applications/notebooks/chat_app


## Step 3 - Customize to use Ollama local LLM (Optional)
- edit chat_app/react_agent/graph.py
    - add this import near top
    ```
    from langchain_openai import ChatOpenAI
    ```
    - search for this line
    ```
    # Initialize the model with tool binding. Change the model or add more tools here.
    model = load_chat_model(configuration.model).bind_tools(TOOLS)
    ```
    - edit to
    ```
    # Initialize the model with tool binding. Change the model or add more tools here.
    # model = load_chat_model(configuration.model).bind_tools(TOOLS)
    model = ChatOpenAI(
        model="llama3.2:1b-instruct-q4_0",
        base_url="http://localhost:11434/v1",
        api_key="not-needed").bind_tools(TOOLS)
    ```

## Step 4 - Pull llama3.2:1b model (Optional)

In [8]:
!ollama pull llama3.2:1b-instruct-q4_0

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling b98b382ad859... 100% ▕████████████████▏ 770 MB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling e9e2eec515fb... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


## Step 5 - setup .env (in chat_app dir)
- OPENAI_API_KEY not needed if using ollama option
```
LANGCHAIN_TRACING_V2=true
LANGCHAIN_PROJECT=005_deploy_lab
LANGCHAIN_API_KEY=xx
TAVILY_API_KEY=xx
OPENAI_API_KEY=xx
```

## Step 6 start local dev env
- open integrated terminal in chat_app folder
- type `langgraph dev`
    - will spawn 
        - API server on localhost
        - Studio UI on LangSmith site
            - login with your creds

## Step 7 Interact via LangGraph SDK
- update to correct port on localhost


In [9]:
# Connect via SDK
url_for_cli_deployment = "http://localhost:2024"

## Streaming Runs
* Each time a client makes a streaming request:
    * The HTTP worker generates a unique run ID
    * The Queue worker begins work on the run
    * During execution, the Queue worker publishes update to Redis
    * The HTTP worker subscribes to updates from Redis for ths run, and returns them to the client

In [10]:
from langchain_core.messages import HumanMessage
from langgraph_sdk import get_client

config = {"configurable": {"user_id": "user1"}}
graph_name = "agent" 
client = get_client(url=url_for_cli_deployment)

thread = await client.threads.create()
user_input = "Who won the last superbowl?"
async for chunk in client.runs.stream(thread["thread_id"], 
                                      graph_name, 
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      config=config,
                                      stream_mode="messages-tuple"):

    if chunk.event == "messages":
        print("".join(data_item['content'] for data_item in chunk.data if 'content' in data_item), end="", flush=True)

[{"title": "The Last 10 Super Bowl Winners - Ticketmaster Blog", "url": "https://blog.ticketmaster.com/super-bowl-winners/", "content": "The Kansas City Chiefs defeated the Philadelphia Eagles to win 38–35, led by a stellar performance from quarterback Patrick Mahomes.", "score": 0.7876867}, {"title": "List of Super Bowl Winners - Topend Sports", "url": "https://www.topendsports.com/events/super-bowl/winners-list.htm", "content": "Super Bowl Winners ; LI, 2017, New England Patriots ; L · 2016, Denver Broncos ; XLIX, 2015, New England Patriots ; XLVIII, 2014, Seattle Seahawks", "score": 0.7746691}, {"title": "Super Bowl Winners and Results | FOX Sports", "url": "https://www.foxsports.com/nfl-super-bowl-winners-and-results", "content": "| III | Miami Orange Bowl | Jan. 12, 1969 | New York Jets 16, Baltimore 7 | | XVIII | Tampa Stadium | Jan. 22, 1984 | Los Angeles Raiders 38, Washington 9 | | XIX | Stanford Stadium | Jan. 20, 1985 | San Francisco 38, Miami 16 | | XXI | Rose Bowl | Jan. 2